# SCohenLab 2D Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## 1. GOAL SETTING ✍

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_ ✅
- robust inference of subcellular objects:
  -  #### 1️⃣. [infer NUCLEI ](./01_infer_nuclei.ipynb) 
  -  #### 2️⃣. [Infer SOMA](./02_infer_soma.ipynb) 🚧 WIP 🚧 (🚨🚨🚨🚨 Steps 3-9 depend on establishing a good solution here.)
  -  #### 3️⃣. [Infer CYTOSOL](./03_infer_cytosol.ipynb)🚧 WIP 🚧
  -  #### 4️⃣. [Infer LYSOSOMES](./04_infer_lysosome.ipynb)
  -  #### 5️⃣. [Infer MITOCHONDRIA](./05_infer_mitochondria.ipynb)
  -  #### 6️⃣. [Infer GOLGI complex](./06_golgi.ipynb)
  -  #### 7️⃣. [Infer PEROXISOMES](./07_peroxisome.ipynb)
  -  #### 8️⃣. [Infer ENDOPLASMIC RETICULUM ](./08_endoplasmic_reticulum.ipynb)
  -  #### 9️⃣. [Infer LB](./09_lipid_bodies.ipynb) 🚧 WIP 🚧





## 2. DATA CREATION
> METHODS:📚📚
> 
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA 🏺 (artifacts)
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## 3. IMAGE PROCESSING
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
A single "optimal" Z slice is chosen for each image for subsequent 2D analysis.
We will perform computational image analysis on the pictures to _segment_ (or _binarize_) the components of interest for measurement.  In other procedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## 4. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Segmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A
# IMPORTS

import  all nescessary packages

we are using `napari` for visualization, and `scipy` `ndimage` and `skimage` for analyzing the image files.  The underlying data format are `numpy` `ndarrays` and tools from  Allen Institute for Cell Science.


In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage import morphology
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import read_input_image, list_image_files
from infer_subc_2d.utils.img import *

%load_ext autoreload
%autoreload 2

viewer = None
# # from infer_subc_2d.bioim.object import BioImObject
# # from infer_subc_2d.transforms.transform import BioImTransform
# # from infer_subc_2d.transforms.pipeline import BioImPipeline
# from infer_subc_2d.utils.file_io import read_input_image, get_raw_meta_data

In [2]:

from aicssegmentation.workflow import WorkflowEngine, WorkflowStep, WorkflowDefinition


# from dataclasses import dataclass
# from typing import List
# from aicssegmentation.workflow import Workflow


# @dataclass
# class Channel:
#     index: int
#     name: str = None

#     @property
#     def display_name(self):
#         if self.name is None or self.name.strip().isspace():
#             return f"Channel {self.index}"

#         return f"Ch{self.index}.  {self.name}"

# @dataclass
# class SegmenterModel:
#     """
#     Main Segmenter plugin model
#     """

#     layers: List[str] = None
#     selected_layer: Layer = None
#     channels: List[str] = None
#     selected_channel: Channel = None
#     workflows: List[str] = None
#     active_workflow: Workflow = None

#     def reset(self):
#         """
#         Reset model state
#         """
#         self.layers = None
#         self.selected_layer = None
#         self.channels = None
#         self.selected_channel = None
#         self.workflows = None
#         self.active_workflow = None

In [3]:
#sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))
# viewer = napari.Viewer()


# Get and load Image for processing
For testing purposes... TODO: build a nice wrapper for this.



Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.

> the `data_path` variable should have the full path to the set of images wrapped in a `Path()`.   Below the path is built in 3 stages
> 1. my user directory "~" plus
> 2. general imaging data directory "Projects/Imaging/data" plus
> 3. "raw" where the linearly unmixed zstacks are

The image "type" is also set by `im_type = ".czi"`


In [4]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# depricate this
# list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]
img_file_list = list_image_files(data_path,im_type)

test_img_name = img_file_list[5]
test_img_name

'/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi'

In [5]:
bioim_image = read_input_image(test_img_name)

img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## CHOOZE Z-SLICE

We need to chooze which Z-slice to analyze for subsequent organelle segmentation.  _A priori_ we expect that the other organelles are ***NOT*** _inside_ of the nucleus, and we want to chooze a Z with minimal overlap between the nucleus and other organelles.  For now we will also assume that a majority of the signals are attributable to a single well marked cell.


In the future we might develop an optimization procedure to resample along an arbitrary plane such that the total florescence signal is maximized and the overlap on nuclei with other organelles.   We may also want to limit ourselves to a single neuron / soma.

Maria Clara's description of how she choose Z-slices:  "I do choose the slice which it contains all the organelle (Nuclei and Golgi are the most difficult one since show high polarity) and that it would contain the highest density of them (you can see trough the slices that some organelle show high density in specific planes)"


In [7]:
# get nuclei and normalize...
# median filter in 2D / convert to float 0-1.   get rid of the "residual"
nuclei = simple_intensity_normalization(img_data[0].copy())

med_filter_size = 4   
# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
nuclei = median_filter_slice_by_slice( 
                                                                nuclei,
                                                                size=med_filter_size  )

gaussian_smoothing_sigma = 1.34
gaussian_smoothing_truncate_range = 3.0
nuclei = image_smoothing_gaussian_slice_by_slice(   nuclei,
                                                                                                    sigma=gaussian_smoothing_sigma,
                                                                                                    truncate_range = gaussian_smoothing_truncate_range
                                                                                                )

#struct_obj = struct_img > filters.threshold_li(struct_img)
threshold_value_log = threshold_li_log(nuclei)

threshold_factor = 0.9 #from cellProfiler
thresh_min = .1
thresh_max = 1.
threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)

struct_obj = nuclei > threshold

###################


In [8]:

# mask everything and find most intense flourescence NOT in the nuclei
ch_to_agg = (1,2,3,4,5,6)

total_florescence = img_data[ch_to_agg,].astype( np.double ).sum(axis=0)
print(total_florescence.shape)
total_ = total_florescence
total_[struct_obj] =0 
optimal_Z = total_.sum(axis=(1,2)).argmax()
optimal_Z


(16, 768, 768)


8

In [9]:
# copy this to base.py for easy import
from typing import Tuple

def _find_optimal_Z(raw_img:np.ndarray, nuc_ch:int, ch_to_agg:Tuple[int]) -> int:
    """
    Procedure to infer NUCLEI from linearly unmixed input.

    Parameters:
    ------------
    raw_img: np.ndarray
        a ch,z,x,y - image containing florescent signal

    nuclei_ch: int
        holds the needed parameters

    nuclei_ch: int
        holds the needed parameters

    Returns:
    -------------
    opt_z:
        the "0ptimal" z-slice which has the most signal intensity for downstream 2D segmentation    
    """

    # median filter in 2D / convert to float 0-1.   get rid of the "residual"
    nuclei = simple_intensity_normalization(raw_img[nuc_ch].copy())

    med_filter_size = 4   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    nuclei = median_filter_slice_by_slice( 
                                                                    nuclei,
                                                                    size=med_filter_size  )

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    nuclei = image_smoothing_gaussian_slice_by_slice(  nuclei,
                                                                                                sigma=gaussian_smoothing_sigma,
                                                                                                truncate_range = gaussian_smoothing_truncate_range
                                                                                                )

    #struct_obj = struct_img > filters.threshold_li(struct_img)
    threshold_value_log = threshold_li_log(nuclei)

    threshold_factor = 0.9 #from cellProfiler
    thresh_min = .1
    thresh_max = 1.
    threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)

    struct_obj = nuclei > threshold
    # find the z with the maximum non-nuclei florescence
    total_florescence = raw_img[ch_to_agg,].astype( np.double ).sum(axis=0)
    total_florescence[struct_obj] =0 
    optimal_Z = total_florescence.sum(axis=(1,2)).argmax()

    return optimal_Z


In [10]:
ch_to_agg = (1,2,3,4,5,6)
nuc_ch = 0

In [11]:
optimal_Z = _find_optimal_Z(img_data, nuc_ch, ch_to_agg) 


In [13]:
optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
optimal_Z

8

In [ ]:

structure_img_smooth = ndi.gaussian_filter( struct_img, sigma=gaussian_smoothing_sigma, mode="nearest", truncate=gaussian_smoothing_truncate_range)



In [11]:
structure_img_median.dtype

dtype('float64')

In [ ]:

#pixels
gaussian_smoothing_sigma = 2.0

gaussian_smoothing_truncate_range = 4.0
# cross product 


#
structure_img_median = ndi.median_filter(struct_img, size = med_filter_size)
structure_img_smooth = ndi.gaussian_filter( struct_img, sigma=gaussian_smoothing_sigma, mode="nearest", truncate=gaussian_smoothing_truncate_range)


Or do segmentation on EVERY slice and then calculate optimal slice _later_?


In [7]:

total_florescence = img_data[0:-2].astype( np.double ).sum(axis=0)
max_proj = ndi.maximum_filter(total_florescence,size = (3,1,1))


max_Z = max_proj.sum(axis=(1,2)).argmax()
im_dat = img_data[:,max_Z,:,:]
im_dat.shape

(8, 768, 768)

In [13]:
tot_organelle_pwr = (img_data[1:-2] ** 2).mean(axis=(2,3)).mean(axis=0)
tot_organelle_pwr

array([13591.60792914, 13716.53311564, 13600.09623244, 13467.81564229,
       13454.94313863, 13532.09359266, 13705.09108209, 13869.04088677,
       14032.77326457, 14099.54194336, 14009.02593418, 13889.99902751,
       13637.38881836, 13343.11465454, 13106.73651937, 12902.12800225])

[<Image layer 'Image [1]' at 0x18043b940>,
 <Image layer 'Image [2]' at 0x180579fd0>,
 <Image layer 'Image [3]' at 0x18064ff40>,
 <Image layer 'Image [4]' at 0x18074efa0>,
 <Image layer 'Image [5]' at 0x1807e9eb0>,
 <Image layer 'Image [6]' at 0x180809fd0>,
 <Image layer 'Image [7]' at 0x18089eeb0>,
 <Image layer 'Image [8]' at 0x180935ee0>]

In [18]:
overlap_pwr[0].shape

(16, 768, 768)

## TODO:  create a napari plugin to interactively find the best slice...

- minimize: overlap_matric {nuclei(ch0), all other organelles (1-6)}
- maximize: signal_pwr (ch 1-6)

In [17]:
viewer.add_image(overlap_pwr[0])

<Image layer 'Image' at 0x1800fd4c0>

---------------------------
Please proceed to 01_infer_nuclei.ipynb


everything below is just testing some speed of different libraries..  

Visualize the raw data file with [napari](https://napari.org)

In [26]:

viewer = napari.view_image(
    img_data,
    channel_axis=0,
    name=channel_names,
    scale=scale
)
viewer.scale_bar.visible = True


In [ ]:
viewer.dims.ndisplay = 3
viewer.camera.angles = (-30, 25, 120)

In [ ]:
##  need to save: 

# output_path, list_of_files
viewer.dims.ndisplay = 2